In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt
from sklearn.metrics import mean_squared_error
import pmdarima
from composition_stats import closure
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os 
  
# current directory 
current_dir = os.getcwd() 
relative_path=os.path.abspath(os.path.join(current_dir, os.pardir))

In [3]:
train_data= pd.DataFrame(pd.read_csv(
    relative_path+"\\age_structure_data\\df_train.csv"))
train_data['Year'] = pd.to_datetime(train_data['Year'])
train_data = train_data.set_index("Year")

test_data= pd.DataFrame(pd.read_csv(
    relative_path+"\\age_structure_data\\df_test.csv"))
test_data['Year'] = pd.to_datetime(test_data['Year'])
test_data = test_data.set_index("Year")

In [4]:
e_ori=train_data['elder']
y_ori=train_data['youth']
k_ori=train_data['kid']

In [5]:
def find_best_pdq(train_data, test_data, d):
    df_search = pd.DataFrame()
    for p in range(6):
            for q in range(6):
                arima_model = ARIMA(train_data, order=(p, d, q)).fit()
                # predict from the position of test data which is length of train data
                prediction = arima_model.predict(
                    start=len(train_data), end=len(train_data)+len(test_data)-1)
                rmse = sqrt(mean_squared_error(test_data, prediction))
                df_search = df_search.append({'pdq_paramters': str(
                    p)+" "+str(d)+" "+str(q), 'rmse': rmse}, ignore_index=True)
    return df_search, test_data, prediction

In [6]:
train_e_1, test_e_1 = train_test_split(e_ori, test_size=0.2, shuffle=False)
train_e_2, test_e_2 = train_test_split(train_e_1, test_size=0.2, shuffle=False)
train_e_3, test_e_3 = train_test_split(train_e_2, test_size=0.15, shuffle=False)
log_ratio_e_best_pdq1, test_data_e_1, prediction_test_e_1=find_best_pdq(train_e_1, test_e_1, 2)
log_ratio_e_best_pdq2, test_data_e_2, prediction_test_e_2=find_best_pdq(train_e_2, test_e_2, 2)
log_ratio_e_best_pdq3, test_data_e_3, prediction_test_e_3=find_best_pdq(train_e_3, test_e_3, 2)

concatenated_1 = pd.concat([log_ratio_e_best_pdq1, log_ratio_e_best_pdq2, log_ratio_e_best_pdq3])
concatenated_1=concatenated_1.groupby('pdq_paramters').mean()
concatenated_1.sort_values('rmse', ascending=True).head()

,rmse
pdq_paramters,
3 2 2,0.000669
2 2 2,0.000731
2 2 3,0.001512
5 2 4,0.001557
3 2 5,0.001656


In [7]:
arima_model_e = ARIMA(e_ori, order=(3, 2, 2)).fit()
prediction_e = arima_model_e.predict(
    start=len(e_ori), end=len(e_ori)+len(test_data)-1)
prediction_e = prediction_e.to_frame()
prediction_e.rename(columns = {'predicted_mean':0}, inplace = True)
final_e = pd.concat([e_ori, prediction_e])

In [8]:
train_y_1, test_y_1 = train_test_split(y_ori, test_size=0.2, shuffle=False)
train_y_2, test_y_2 = train_test_split(train_y_1, test_size=0.2, shuffle=False)
train_y_3, test_y_3 = train_test_split(train_y_2, test_size=0.15, shuffle=False)
log_ratio_y_best_pdq1, test_data_y_1, prediction_test_y_1=find_best_pdq(train_y_1, test_y_1, 1)
log_ratio_y_best_pdq2, test_data_y_2, prediction_test_y_2=find_best_pdq(train_y_2, test_y_2, 1)
log_ratio_y_best_pdq3, test_data_y_3, prediction_test_y_3=find_best_pdq(train_y_3, test_y_3, 1)

concatenated_1 = pd.concat([log_ratio_y_best_pdq1, log_ratio_y_best_pdq2, log_ratio_y_best_pdq3])
concatenated_1=concatenated_1.groupby('pdq_paramters').mean()
concatenated_1.sort_values('rmse', ascending=True).head()

,rmse
pdq_paramters,
4 1 3,0.002114
1 1 0,0.002727
2 1 0,0.002859
1 1 2,0.003076
4 1 2,0.003088


In [9]:
arima_model_y = ARIMA(y_ori, order=(4, 1, 3)).fit()
prediction_y = arima_model_y.predict(
    start=len(y_ori), end=len(y_ori)+len(test_data)-1)
prediction_y = prediction_y.to_frame()
prediction_y.rename(columns = {'predicted_mean':0}, inplace = True)
final_y = pd.concat([y_ori, prediction_y])

In [10]:
train_k_1, test_k_1 = train_test_split(k_ori, test_size=0.2, shuffle=False)
train_k_2, test_k_2 = train_test_split(train_k_1, test_size=0.2, shuffle=False)
train_k_3, test_k_3 = train_test_split(train_k_2, test_size=0.15, shuffle=False)
log_ratio_k_best_pdq1, test_data_k_1, prediction_test_k_1=find_best_pdq(train_k_1, test_k_1, 1)
log_ratio_k_best_pdq2, test_data_k_2, prediction_test_k_2=find_best_pdq(train_k_2, test_k_2, 1)
log_ratio_k_best_pdq3, test_data_k_3, prediction_test_k_3=find_best_pdq(train_k_3, test_k_3, 1)

concatenated_1 = pd.concat([log_ratio_k_best_pdq1, log_ratio_k_best_pdq2, log_ratio_k_best_pdq3])
concatenated_1=concatenated_1.groupby('pdq_paramters').mean()
concatenated_1.sort_values('rmse', ascending=True).head()

,rmse
pdq_paramters,
3 1 3,0.003076
4 1 3,0.003140
5 1 3,0.003217
2 1 4,0.003315
3 1 4,0.003359


In [11]:
arima_model_k = ARIMA(k_ori, order=(3, 1, 2)).fit()
prediction_k = arima_model_k.predict(
    start=len(k_ori), end=len(k_ori)+len(test_data)-1)
prediction_k = prediction_k.to_frame()
prediction_k.rename(columns = {'predicted_mean':0}, inplace = True)
final_k = pd.concat([k_ori, prediction_k])

In [12]:
final_e.rename(columns = {0:'e'}, inplace = True)
final_y.rename(columns = {0:'y'}, inplace = True)
final_k.rename(columns = {0:'k'}, inplace = True)

In [13]:
full = pd.concat([final_e, final_y,final_k],axis=1)

In [14]:
data_proportion = closure(full)

In [15]:
def set_col(df):
    df = pd.DataFrame(df, columns=[
    'elder', 'youth', 'kid'])
    df['Year'] = df.index + 1964
    df['Year'] = pd.to_datetime(df['Year'], format='%Y')
    df = df.set_index("Year")
    return df

In [16]:
proportion = set_col(data_proportion)

In [17]:
df_actual=proportion[(proportion.index< '2010-01-01')]
df_forecast=proportion[(proportion.index> '2009-01-01')]


In [18]:
proportion_actual_1=closure(pd.concat([test_data_e_1.to_frame(), test_data_y_1.to_frame(),  test_data_k_1.to_frame()],axis=1))
proportion_actual_2=closure(pd.concat([test_data_e_2.to_frame(), test_data_y_2.to_frame(), test_data_k_2.to_frame()],axis=1))
proportion_actual_3=closure(pd.concat([test_data_e_3.to_frame(), test_data_y_3.to_frame(), test_data_k_3.to_frame()],axis=1))
proportion_forecast_1=closure(pd.concat([prediction_test_e_1.to_frame(), prediction_test_y_1.to_frame(), prediction_test_k_1.to_frame()],axis=1))
proportion_forecast_2=closure(pd.concat([prediction_test_e_2.to_frame(), prediction_test_y_2.to_frame(),prediction_test_k_2.to_frame()],axis=1))
proportion_forecast_3=closure(pd.concat([prediction_test_e_3.to_frame(), prediction_test_y_3.to_frame(),prediction_test_k_3.to_frame()],axis=1))

In [19]:
rmse1 = sqrt(mean_squared_error(proportion_actual_1, proportion_forecast_1))
rmse2 = sqrt(mean_squared_error(proportion_actual_2, proportion_forecast_2))
rmse3 = sqrt(mean_squared_error(proportion_actual_3, proportion_forecast_3))

rmse_cv=(rmse1+rmse2+rmse3)/3
rmse_cv

0.004692927491535809

In [20]:
rmse_final = sqrt(mean_squared_error(df_forecast, test_data))
rmse_final

0.008330358548700201

In [22]:
df_forecast.to_csv(relative_path+"\\age_structure_data\\grid_base_forecast.csv")